<a href="https://colab.research.google.com/github/Tony607/object_detection_demo/blob/master/tensorflow_object_detection_training_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [How to train an object detection model easy for free](https://www.dlology.com/blog/how-to-train-an-object-detection-model-easy-for-free/) | DLology Blog

## Configs and Hyperparameters

Support a variety of models, you can find more pretrained model from [Tensorflow detection model zoo: COCO-trained models](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md#coco-trained-models), as well as their pipline config files in [object_detection/samples/configs/](https://github.com/tensorflow/models/tree/master/research/object_detection/samples/configs).

In [1]:
# If you forked the repository, you can replace the link.
repo_url = 'https://github.com/Tony607/object_detection_demo'

# Number of training steps.
num_steps = 20000  # 200000

# Number of evaluation steps.
num_eval_steps = 50

MODELS_CONFIG = {
    'ssd_mobilenet_v2': {
        'model_name': 'ssd_mobilenet_v2_coco_2018_03_29',
        'pipeline_file': 'ssd_mobilenet_v2_coco.config',
        'batch_size': 12
    },
    'faster_rcnn_inception_v2': {
        'model_name': 'faster_rcnn_inception_v2_coco_2018_01_28',
        'pipeline_file': 'faster_rcnn_inception_v2_pets.config',
        'batch_size': 12
    },
    'rfcn_resnet101': {
        'model_name': 'rfcn_resnet101_coco_2018_01_28',
        'pipeline_file': 'rfcn_resnet101_pets.config',
        'batch_size': 8
    }
}

# Pick the model you want to use
# Select a model in `MODELS_CONFIG`.
selected_model = 'ssd_mobilenet_v2'

# Name of the object detection model to use.
MODEL = MODELS_CONFIG[selected_model]['model_name']

# Name of the pipline file in tensorflow object detection API.
pipeline_file = MODELS_CONFIG[selected_model]['pipeline_file']

# Training batch size fits in Colabe's Tesla K80 GPU memory for selected model.
batch_size = MODELS_CONFIG[selected_model]['batch_size']

## Clone the `object_detection_demo` repository or your fork.

## Install required packages

In [2]:
# %cd /home/ghost/Desktop/experiment/

# !git clone https://github.com/tensorflow/models.git

# !apt-get install -qq protobuf-compiler python-pil python-lxml python-tk

# !pip install -q Cython contextlib2 pillow lxml matplotlib

# !pip install -q pycocotools

# %cd /home/ghost/Desktop/experiment/models/research

# ! python setup.py build

# ! python setup.py install

# %cd /home/ghost/Desktop/experiment/models/research/slim

# ! pip install -e .

%cd /home/ghost/Desktop/experiment/models/research

!protoc object_detection/protos/*.proto --python_out=.

!export PYTHONPATH=$PYTHONPATH:/home/ghost/Desktop/experiment/models/research/:/home/ghost/Desktop/experiment/models/research/slim
        
! ls

!python object_detection/builders/model_builder_test.py

/home/ghost/Desktop/experiment/models/research
a3c_blogpost			  marco
adversarial_crypto		  maskgan
adversarial_logit_pairing	  morph_net
adversarial_text		  namignizer
adv_imagenet_models		  neural_gpu
astronet			  neural_programmer
attention_ocr			  next_frame_prediction
audioset			  ngrok
autoaugment			  ngrok-stable-linux-amd64.zip
autoencoder			  ngrok-stable-linux-amd64.zip.1
brain_coder			  nst_blogpost
build				  object_detection
cognitive_mapping_and_planning	  object_detection.egg-info
cognitive_planning		  pcl_rl
compression			  pretrained_model
cvt_text			  ptn
deep_contextual_bandits		  qa_kg
deeplab				  README.md
deep_speech			  real_nvp
delf				  rebar
differential_privacy		  sentiment_analysis
dist				  seq2species
domain_adaptation		  setup.py
efficient-hrl			  skip_thoughts
feelvos				  slim
fivo				  steve
global_objectives		  street
im2txt				  struct2depth
inception			  swivel
keypointnet			  syntaxnet
learned_optimizer		  tcn
learning_to_remember_rare_events  ten

## Prepare `tfrecord` files

Use the following scripts to generate the `tfrecord` files.
```bash
# Convert train folder annotation xml files to a single csv file,
# generate the `label_map.pbtxt` file to `data/` directory as well.
python xml_to_csv.py -i data/images/train -o data/annotations/train_labels.csv -l data/annotations

# Convert test folder annotation xml files to a single csv.
python xml_to_csv.py -i data/images/test -o data/annotations/test_labels.csv

# Generate `train.record`
python generate_tfrecord.py --csv_input=data/annotations/train_labels.csv --output_path=data/annotations/train.record --img_path=data/images/train --label_map data/annotations/label_map.pbtxt

# Generate `test.record`
python generate_tfrecord.py --csv_input=data/annotations/test_labels.csv --output_path=data/annotations/test.record --img_path=data/images/test --label_map data/annotations/label_map.pbtxt
```

In [3]:
repo_dir_path = "/home/ghost/Desktop/object_detection_demo/"

%cd {repo_dir_path}

# Convert train folder annotation xml files to a single csv file,
# generate the `label_map.pbtxt` file to `data/` directory as well.
!python xml_to_csv.py -i data/images/train -o data/annotations/train_labels.csv -l data/annotations

# Convert test folder annotation xml files to a single csv.
!python xml_to_csv.py -i data/images/test -o data/annotations/test_labels.csv

# Generate `train.record`
!python generate_tfrecord.py --csv_input=data/annotations/train_labels.csv --output_path=data/annotations/train.record --img_path=data/images/train --label_map data/annotations/label_map.pbtxt

# Generate `test.record`
!python generate_tfrecord.py --csv_input=data/annotations/test_labels.csv --output_path=data/annotations/test.record --img_path=data/images/test --label_map data/annotations/label_map.pbtxt

/home/ghost/Desktop/object_detection_demo
Successfully converted xml to csv.
Generate `data/annotations/label_map.pbtxt`
Successfully converted xml to csv.


W0208 19:04:54.247012 140549989967680 module_wrapper.py:139] From generate_tfrecord.py:107: The name tf.python_io.TFRecordWriter is deprecated. Please use tf.io.TFRecordWriter instead.


W0208 19:04:54.258589 140549989967680 module_wrapper.py:139] From /home/ghost/anaconda3/envs/dtoxd/lib/python3.7/site-packages/object_detection-0.1-py3.7.egg/object_detection/utils/label_map_util.py:138: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

Successfully created the TFRecords: /home/ghost/Desktop/object_detection_demo/data/annotations/train.record


W0208 19:04:57.048276 140558865352512 module_wrapper.py:139] From generate_tfrecord.py:107: The name tf.python_io.TFRecordWriter is deprecated. Please use tf.io.TFRecordWriter instead.


W0208 19:04:57.054518 140558865352512 module_wrapper.py:139] From /home/ghos

In [4]:
test_record_fname = '/home/ghost/Desktop/object_detection_demo/data/annotations/test.record'
train_record_fname = '/home/ghost/Desktop/object_detection_demo/data/annotations/train.record'
label_map_pbtxt_fname = '/home/ghost/Desktop/object_detection_demo/data/annotations/label_map.pbtxt'

## Download base model

In [5]:
%cd /home/ghost/Desktop/experiment/models/research

import os
import shutil
import glob
import urllib.request
import tarfile
MODEL_FILE = MODEL + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
DEST_DIR = '/home/ghost/Desktop/experiment/models/research/pretrained_model'

if not (os.path.exists(MODEL_FILE)):
    urllib.request.urlretrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

os.remove(MODEL_FILE)
if (os.path.exists(DEST_DIR)):
    shutil.rmtree(DEST_DIR)
os.rename(MODEL, DEST_DIR)

/home/ghost/Desktop/experiment/models/research


In [6]:
!echo {DEST_DIR}
!ls -alh {DEST_DIR}

/home/ghost/Desktop/experiment/models/research/pretrained_model
total 135M
drwxr-xr-x  3 ghost ghost 4.0K Mar 30  2018 .
drwxr-xr-x 72 ghost ghost 4.0K Feb  8 19:06 ..
-rw-r--r--  1 ghost ghost   77 Mar 30  2018 checkpoint
-rw-r--r--  1 ghost ghost  67M Mar 30  2018 frozen_inference_graph.pb
-rw-r--r--  1 ghost ghost  65M Mar 30  2018 model.ckpt.data-00000-of-00001
-rw-r--r--  1 ghost ghost  15K Mar 30  2018 model.ckpt.index
-rw-r--r--  1 ghost ghost 3.4M Mar 30  2018 model.ckpt.meta
-rw-r--r--  1 ghost ghost 4.2K Mar 30  2018 pipeline.config
drwxr-xr-x  3 ghost ghost 4.0K Mar 30  2018 saved_model


In [7]:
fine_tune_checkpoint = os.path.join(DEST_DIR, "model.ckpt")
fine_tune_checkpoint

'/home/ghost/Desktop/experiment/models/research/pretrained_model/model.ckpt'

## Configuring a Training Pipeline

In [8]:
import os
pipeline_fname = os.path.join('/home/ghost/Desktop/experiment/models/research/object_detection/samples/configs/', pipeline_file)

assert os.path.isfile(pipeline_fname), '`{}` not exist'.format(pipeline_fname)

In [9]:
def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())

In [10]:
import re

num_classes = get_num_classes(label_map_pbtxt_fname)
with open(pipeline_fname) as f:
    s = f.read()
with open(pipeline_fname, 'w') as f:
    
    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
    
    # tfrecord files train and test.
    s = re.sub(
        '(input_path: ".*?)(train.record)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(val.record)(.*?")', 'input_path: "{}"'.format(test_record_fname), s)

    # label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)
    
    # Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)
    f.write(s)

In [11]:
!cat {pipeline_fname}

# SSD with Mobilenet v2 configuration for MSCOCO Dataset.
# Users should configure the fine_tune_checkpoint field in the train config as
# well as the label_map_path and input_path fields in the train_input_reader and
# eval_input_reader. Search for "PATH_TO_BE_CONFIGURED" to find the fields that
# should be configured.

model {
  ssd {
    num_classes: 4
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    anchor_generator {
      ssd_anchor_generator {
        num_layers: 6
        min_scale: 0.2
        max_scale: 0.95
        aspect_ratios: 1.0
        aspect_ratios: 2.0
        aspect_

In [12]:
!pwd
model_dir = 'training/'
# Optionally remove content in output model directory to fresh start.
!rm -rf {model_dir}
os.makedirs(model_dir, exist_ok=True)

/home/ghost/Desktop/experiment/models/research


## Run Tensorboard(Optional)

### Get Tensorboard link

## Train the model

In [13]:
!pwd

%cd /home/ghost/Desktop/experiment/

! git clone https://github.com/cocodataset/cocoapi.git

%cd cocoapi/PythonAPI

!make

!python setup.py build

!python setup.py install

/home/ghost/Desktop/experiment/models/research
/home/ghost/Desktop/experiment
fatal: destination path 'cocoapi' already exists and is not an empty directory.
/home/ghost/Desktop/experiment/cocoapi/PythonAPI
python setup.py build_ext --inplace
running build_ext
skipping 'pycocotools/_mask.c' Cython extension (up-to-date)
copying build/lib.linux-x86_64-3.7/pycocotools/_mask.cpython-37m-x86_64-linux-gnu.so -> pycocotools
rm -rf build
running build
running build_py
creating build
creating build/lib.linux-x86_64-3.7
creating build/lib.linux-x86_64-3.7/pycocotools
copying pycocotools/mask.py -> build/lib.linux-x86_64-3.7/pycocotools
copying pycocotools/coco.py -> build/lib.linux-x86_64-3.7/pycocotools
copying pycocotools/__init__.py -> build/lib.linux-x86_64-3.7/pycocotools
copying pycocotools/cocoeval.py -> build/lib.linux-x86_64-3.7/pycocotools
running build_ext
skipping 'pycocotools/_mask.c' Cython extension (up-to-date)
building 'pycocotools._mask' extension
creating build/common
creatin

removing 'build/bdist.linux-x86_64/egg' (and everything under it)
Processing pycocotools-2.0-py3.7-linux-x86_64.egg
removing '/home/ghost/anaconda3/envs/dtoxd/lib/python3.7/site-packages/pycocotools-2.0-py3.7-linux-x86_64.egg' (and everything under it)
creating /home/ghost/anaconda3/envs/dtoxd/lib/python3.7/site-packages/pycocotools-2.0-py3.7-linux-x86_64.egg
Extracting pycocotools-2.0-py3.7-linux-x86_64.egg to /home/ghost/anaconda3/envs/dtoxd/lib/python3.7/site-packages
pycocotools 2.0 is already the active version in easy-install.pth

Installed /home/ghost/anaconda3/envs/dtoxd/lib/python3.7/site-packages/pycocotools-2.0-py3.7-linux-x86_64.egg
Processing dependencies for pycocotools==2.0
Searching for matplotlib==3.1.1
Best match: matplotlib 3.1.1
Adding matplotlib 3.1.1 to easy-install.pth file

Using /home/ghost/anaconda3/envs/dtoxd/lib/python3.7/site-packages
Searching for Cython==0.29.14
Best match: Cython 0.29.14
Adding Cython 0.29.14 to easy-install.pth file
Installing cygdb scr

In [ ]:
!python /home/ghost/Desktop/experiment/models/research/object_detection/model_main.py \
    --pipeline_config_path={pipeline_fname} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --num_eval_steps={num_eval_steps}

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



W0208 19:08:18.794018 140033311766336 module_wrapper.py:139] From /home/ghost/anaconda3/envs/dtoxd/lib/python3.7/site-packages/object_detection-0.1-py3.7.egg/object_detection/utils/config_util.py:102: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0208 19:08:18.797579 140033311766336 module_wrapper.py:139] From /home/ghost/anaconda3/envs/dtoxd/lib/python3.7/site-packages/object_detection-0.1-py3.7.egg/object_detection/model_lib.py:628: The name tf.logging.warning is deprecated. Please use tf.compat.v1.logging.warning instead.

W0208 19:08:18.797749 140033311766336 model_lib.py:629] Forced numbe

Instructions for updating:
Use `tf.data.experimental.parallel_interleave(...)`.
W0208 19:08:18.838284 140033311766336 deprecation.py:323] From /home/ghost/anaconda3/envs/dtoxd/lib/python3.7/site-packages/object_detection-0.1-py3.7.egg/object_detection/builders/dataset_builder.py:86: parallel_interleave (from tensorflow.contrib.data.python.ops.interleave_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.data.experimental.parallel_interleave(...)`.
Instructions for updating:
Use `tf.data.Dataset.interleave(map_func, cycle_length, block_length, num_parallel_calls=tf.data.experimental.AUTOTUNE)` instead. If sloppy execution is desired, use `tf.data.Options.experimental_determinstic`.
W0208 19:08:18.838500 140033311766336 deprecation.py:323] From /home/ghost/anaconda3/envs/dtoxd/lib/python3.7/site-packages/tensorflow_core/contrib/data/python/ops/interleave_ops.py:77: parallel_interleave (from tensorflow.python.data.experimental.ops.interleave_ops


W0208 19:08:40.168653 140033311766336 module_wrapper.py:139] From /home/ghost/anaconda3/envs/dtoxd/lib/python3.7/site-packages/object_detection-0.1-py3.7.egg/object_detection/meta_architectures/ssd_meta_arch.py:589: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.


W0208 19:08:40.168960 140033311766336 module_wrapper.py:139] From /home/ghost/anaconda3/envs/dtoxd/lib/python3.7/site-packages/object_detection-0.1-py3.7.egg/object_detection/meta_architectures/ssd_meta_arch.py:597: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

Instructions for updating:
Please use `layer.__call__` method instead.
W0208 19:08:40.171188 140033311766336 deprecation.py:323] From /home/ghost/anaconda3/envs/dtoxd/lib/python3.7/site-packages/tensorflow_core/contrib/layers/python/layers/layers.py:1057: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updat


W0208 19:08:46.545308 140033311766336 module_wrapper.py:139] From /home/ghost/anaconda3/envs/dtoxd/lib/python3.7/site-packages/object_detection-0.1-py3.7.egg/object_detection/meta_architectures/ssd_meta_arch.py:1275: The name tf.get_collection is deprecated. Please use tf.compat.v1.get_collection instead.


W0208 19:08:46.547931 140033311766336 module_wrapper.py:139] From /home/ghost/anaconda3/envs/dtoxd/lib/python3.7/site-packages/object_detection-0.1-py3.7.egg/object_detection/model_lib.py:380: The name tf.train.get_or_create_global_step is deprecated. Please use tf.compat.v1.train.get_or_create_global_step instead.


W0208 19:08:46.548311 140033311766336 module_wrapper.py:139] From /home/ghost/anaconda3/envs/dtoxd/lib/python3.7/site-packages/object_detection-0.1-py3.7.egg/object_detection/utils/learning_schedules.py:66: The name tf.train.exponential_decay is deprecated. Please use tf.compat.v1.train.exponential_decay instead.


W0208 19:08:46.561367 140033311766336 module_wrapper.p

INFO:tensorflow:global_step/sec: 0.296784
I0208 19:15:10.212962 140033311766336 basic_session_run_hooks.py:692] global_step/sec: 0.296784
INFO:tensorflow:loss = 7.6305714, step = 101 (336.945 sec)
I0208 19:15:10.213797 140033311766336 basic_session_run_hooks.py:260] loss = 7.6305714, step = 101 (336.945 sec)
INFO:tensorflow:Saving checkpoints for 175 into training/model.ckpt.
I0208 19:19:21.533400 140033311766336 basic_session_run_hooks.py:606] Saving checkpoints for 175 into training/model.ckpt.
INFO:tensorflow:Calling model_fn.
I0208 19:19:23.945573 140033311766336 estimator.py:1148] Calling model_fn.
INFO:tensorflow:depth of additional conv before box predictor: 0
I0208 19:19:26.590701 140033311766336 convolutional_box_predictor.py:151] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0208 19:19:26.627496 140033311766336 convolutional_box_predictor.py:151] depth of additional conv before box predictor: 0
INFO:tensorfl

...

DONE (t=0.14s).
Accumulating evaluation results...
DONE (t=0.05s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.249
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.652
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.141
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.472
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.254
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.300
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.430
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.434
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.533
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= l

INFO:tensorflow:global_step/sec: 0.288642
I0208 20:00:48.056188 140033311766336 basic_session_run_hooks.py:692] global_step/sec: 0.288642
INFO:tensorflow:loss = 4.2014136, step = 901 (346.449 sec)
I0208 20:00:48.056898 140033311766336 basic_session_run_hooks.py:260] loss = 4.2014136, step = 901 (346.449 sec)
INFO:tensorflow:global_step/sec: 0.303299
I0208 20:06:17.764319 140033311766336 basic_session_run_hooks.py:692] global_step/sec: 0.303299
INFO:tensorflow:loss = 4.68996, step = 1001 (329.708 sec)
I0208 20:06:17.765171 140033311766336 basic_session_run_hooks.py:260] loss = 4.68996, step = 1001 (329.708 sec)
INFO:tensorflow:Saving checkpoints for 1059 into training/model.ckpt.
I0208 20:09:28.565161 140033311766336 basic_session_run_hooks.py:606] Saving checkpoints for 1059 into training/model.ckpt.
INFO:tensorflow:Calling model_fn.
I0208 20:09:30.744916 140033311766336 estimator.py:1148] Calling model_fn.
INFO:tensorflow:depth of additional conv before box predictor: 0
I0208 20:09:33

INFO:tensorflow:depth of additional conv before box predictor: 0
I0208 20:19:35.484005 140033311766336 convolutional_box_predictor.py:151] depth of additional conv before box predictor: 0
INFO:tensorflow:Done calling model_fn.
I0208 20:19:37.038607 140033311766336 estimator.py:1150] Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-02-08T20:19:37Z
I0208 20:19:37.055806 140033311766336 evaluation.py:255] Starting evaluation at 2020-02-08T20:19:37Z
INFO:tensorflow:Graph was finalized.
I0208 20:19:37.486187 140033311766336 monitored_session.py:240] Graph was finalized.
INFO:tensorflow:Restoring parameters from training/model.ckpt-1237
I0208 20:19:37.487326 140033311766336 saver.py:1284] Restoring parameters from training/model.ckpt-1237
INFO:tensorflow:Running local_init_op.
I0208 20:19:39.244101 140033311766336 session_manager.py:500] Running local_init_op.
INFO:tensorflow:Done running local_init_op.
I0208 20:19:39.401094 140033311766336 session_manager.py:502] Done runn

INFO:tensorflow:Finished evaluation at 2020-02-08-20:29:47
I0208 20:29:47.591343 140033311766336 evaluation.py:275] Finished evaluation at 2020-02-08-20:29:47
INFO:tensorflow:Saving dict for global step 1412: DetectionBoxes_Precision/mAP = 0.1496345, DetectionBoxes_Precision/mAP (large) = 0.17094807, DetectionBoxes_Precision/mAP (medium) = 0.0036533677, DetectionBoxes_Precision/mAP (small) = -1.0, DetectionBoxes_Precision/mAP@.50IOU = 0.34067315, DetectionBoxes_Precision/mAP@.75IOU = 0.052383393, DetectionBoxes_Recall/AR@1 = 0.13572985, DetectionBoxes_Recall/AR@10 = 0.23028323, DetectionBoxes_Recall/AR@100 = 0.29607844, DetectionBoxes_Recall/AR@100 (large) = 0.29527777, DetectionBoxes_Recall/AR@100 (medium) = 0.25, DetectionBoxes_Recall/AR@100 (small) = -1.0, Loss/classification_loss = 8.528157, Loss/localization_loss = 2.1922753, Loss/regularization_loss = 0.2524319, Loss/total_loss = 10.972863, global_step = 1412, learning_rate = 0.004, loss = 10.972863
I0208 20:29:47.591596 14003331

INFO:tensorflow:global_step/sec: 0.289976
I0208 20:40:25.825999 140033311766336 basic_session_run_hooks.py:692] global_step/sec: 0.289976
INFO:tensorflow:loss = 3.8923295, step = 1601 (344.856 sec)
I0208 20:40:25.826720 140033311766336 basic_session_run_hooks.py:260] loss = 3.8923295, step = 1601 (344.856 sec)
INFO:tensorflow:global_step/sec: 0.302442
I0208 20:45:56.467525 140033311766336 basic_session_run_hooks.py:692] global_step/sec: 0.302442
INFO:tensorflow:loss = 3.8252072, step = 1701 (330.642 sec)
I0208 20:45:56.468236 140033311766336 basic_session_run_hooks.py:260] loss = 3.8252072, step = 1701 (330.642 sec)
INFO:tensorflow:Saving checkpoints for 1766 into training/model.ckpt.
I0208 20:49:36.023094 140033311766336 basic_session_run_hooks.py:606] Saving checkpoints for 1766 into training/model.ckpt.
INFO:tensorflow:Calling model_fn.
I0208 20:49:38.213791 140033311766336 estimator.py:1148] Calling model_fn.
INFO:tensorflow:depth of additional conv before box predictor: 0
I0208 20

In [ ]:
!ls {model_dir}

In [ ]:
# Legacy way of training(also works).
# !python /content/models/research/object_detection/legacy/train.py --logtostderr --train_dir={model_dir} --pipeline_config_path={pipeline_fname}

## Exporting a Trained Inference Graph
Once your training job is complete, you need to extract the newly trained inference graph, which will be later used to perform the object detection. This can be done as follows:

In [ ]:
import re
import numpy as np

output_directory = './fine_tuned_model'

lst = os.listdir(model_dir)
lst = [l for l in lst if 'model.ckpt-' in l and '.meta' in l]
steps=np.array([int(re.findall('\d+', l)[0]) for l in lst])
last_model = lst[steps.argmax()].replace('.meta', '')

last_model_path = os.path.join(model_dir, last_model)
print(last_model_path)
!python /home/ghost/Desktop/experiment/models/research/object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path={pipeline_fname} \
    --output_directory={output_directory} \
    --trained_checkpoint_prefix={last_model_path}

In [ ]:
!ls {output_directory}

## Download the model `.pb` file

In [ ]:
import os

pb_fname = os.path.join(os.path.abspath(output_directory), "frozen_inference_graph.pb")
assert os.path.isfile(pb_fname), '`{}` not exist'.format(pb_fname)

In [ ]:
!ls -alh {pb_fname}

### Option1 : upload the `.pb` file to your Google Drive
Then download it from your Google Drive to local file system.

During this step, you will be prompted to enter the token.

In [ ]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once in a notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

fname = os.path.basename(pb_fname)
# Create & upload a text file.
uploaded = drive.CreateFile({'title': fname})
uploaded.SetContentFile(pb_fname)
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

### Option2 :  Download the `.pb` file directly to your local file system
This method may not be stable when downloading large files like the model `.pb` file. Try **option 1** instead if not working.

In [ ]:
from google.colab import files
files.download(pb_fname)

### Download the `label_map.pbtxt` file

In [ ]:
from google.colab import files
files.download(label_map_pbtxt_fname)

### Download the modified pipline file
If you plan to use OpenVINO toolkit to convert the `.pb` file to inference faster on Intel's hardware (CPU/GPU, Movidius, etc.)

In [ ]:
files.download(pipeline_fname)

In [ ]:
# !tar cfz fine_tuned_model.tar.gz fine_tuned_model
# from google.colab import files
# files.download('fine_tuned_model.tar.gz')

## Run inference test
Test with images in repository `object_detection_demo/test` directory.

In [ ]:
import os
import glob

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = pb_fname

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = label_map_pbtxt_fname

# If you want to test the code with your images, just add images files to the PATH_TO_TEST_IMAGES_DIR.
PATH_TO_TEST_IMAGES_DIR =  os.path.join(repo_dir_path, "test")

assert os.path.isfile(pb_fname)
assert os.path.isfile(PATH_TO_LABELS)
TEST_IMAGE_PATHS = glob.glob(os.path.join(PATH_TO_TEST_IMAGES_DIR, "*.*"))
assert len(TEST_IMAGE_PATHS) > 0, 'No image found in `{}`.'.format(PATH_TO_TEST_IMAGES_DIR)
print(TEST_IMAGE_PATHS)

In [ ]:
%cd /home/ghost/Desktop/experiment/models/research/object_detection

import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops


# This is needed to display the images.
%matplotlib inline


from object_detection.utils import label_map_util

from object_detection.utils import visualization_utils as vis_util


detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')


label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(
    label_map, max_num_classes=num_classes, use_display_name=True)
category_index = label_map_util.create_category_index(categories)


def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)


def run_inference_for_single_image(image, graph):
    with graph.as_default():
        with tf.Session() as sess:
            # Get handles to input and output tensors
            ops = tf.get_default_graph().get_operations()
            all_tensor_names = {
                output.name for op in ops for output in op.outputs}
            tensor_dict = {}
            for key in [
                'num_detections', 'detection_boxes', 'detection_scores',
                'detection_classes', 'detection_masks'
            ]:
                tensor_name = key + ':0'
                if tensor_name in all_tensor_names:
                    tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
                        tensor_name)
            if 'detection_masks' in tensor_dict:
                # The following processing is only for single image
                detection_boxes = tf.squeeze(
                    tensor_dict['detection_boxes'], [0])
                detection_masks = tf.squeeze(
                    tensor_dict['detection_masks'], [0])
                # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
                real_num_detection = tf.cast(
                    tensor_dict['num_detections'][0], tf.int32)
                detection_boxes = tf.slice(detection_boxes, [0, 0], [
                                           real_num_detection, -1])
                detection_masks = tf.slice(detection_masks, [0, 0, 0], [
                                           real_num_detection, -1, -1])
                detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                    detection_masks, detection_boxes, image.shape[0], image.shape[1])
                detection_masks_reframed = tf.cast(
                    tf.greater(detection_masks_reframed, 0.5), tf.uint8)
                # Follow the convention by adding back the batch dimension
                tensor_dict['detection_masks'] = tf.expand_dims(
                    detection_masks_reframed, 0)
            image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

            # Run inference
            output_dict = sess.run(tensor_dict,
                                   feed_dict={image_tensor: np.expand_dims(image, 0)})

            # all outputs are float32 numpy arrays, so convert types as appropriate
            output_dict['num_detections'] = int(
                output_dict['num_detections'][0])
            output_dict['detection_classes'] = output_dict[
                'detection_classes'][0].astype(np.uint8)
            output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
            output_dict['detection_scores'] = output_dict['detection_scores'][0]
            if 'detection_masks' in output_dict:
                output_dict['detection_masks'] = output_dict['detection_masks'][0]
    return output_dict


for image_path in TEST_IMAGE_PATHS:
    image = Image.open(image_path)
    # the array based representation of the image will be used later in order to prepare the
    # result image with boxes and labels on it.
    image_np = load_image_into_numpy_array(image)
    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
    image_np_expanded = np.expand_dims(image_np, axis=0)
    # Actual detection.
    output_dict = run_inference_for_single_image(image_np, detection_graph)
    # Visualization of the results of a detection.
    vis_util.visualize_boxes_and_labels_on_image_array(
        image_np,
        output_dict['detection_boxes'],
        output_dict['detection_classes'],
        output_dict['detection_scores'],
        category_index,
        instance_masks=output_dict.get('detection_masks'),
        use_normalized_coordinates=True,
        line_thickness=8)
    plt.figure(figsize=IMAGE_SIZE)
    plt.imshow(image_np)

In [ ]:
plt.imshow(image_np)